# Rapids KMeans

### Loaded with the MNIST dataset, with TensorFlow and Keras.

In [1]:
import sys
!nvidia-smi

Tue Nov  2 14:43:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:10.0 Off |                  N/A |
| 26%   30C    P8    15W / 250W |    581MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from cuml.dask.cluster import KMeans
from keras.datasets import mnist
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import time
import dask_cudf
import cudf
import cupy

#Load Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train_reshaped = x_train.reshape((60000, 784))
pca = PCA(2)

print("Number of training images: " + str(len(x_train)))
print("Number of test images: " + str(len(x_test)))
print("Number of unique classes: " + str(len(np.unique(y_train))))

#Transform the data
pca_transformed = pca.fit_transform(x_train_reshaped)
et_eller_andet = pca_transformed

def np2cudf(df):
    # convert numpy array to cuDF dataframe
    df = pd.DataFrame({'fea%d'%i:df[:,i] for i in range(df.shape[1])})
    pdf = cudf.DataFrame()
    for c,column in enumerate(df):
      pdf[str(c)] = df[column]
    return pdf

b = np2cudf(pca_transformed)
csv_cuda_splitted = dask_cudf.from_cudf(b, npartitions=10)

Number of training images: 60000
Number of test images: 10000
Number of unique classes: 10


In [4]:
# Dask cuda imports
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
cuda_client = Client("10.42.0.165:8786")
cuda_client

/opt/conda/envs/rapids/lib/python3.7/site-packages/distributed/client.py:1123: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| numpy   | 1.19.5         | 1.21.2         | 1.21.2         |
| python  | 3.7.10.final.0 | 3.8.12.final.0 | 3.8.12.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://10.42.0.165:8786' processes=3 threads=3, memory=5.59 GiB>

In [6]:
t_start = time.time()

#Initialize the class object
kmeans = KMeans(n_clusters=10, client=cuda_client, random_state=0)

#predict the labels of clusters.
label = kmeans.fit_predict(csv_cuda_splitted)

t_stop = time.time()

t_final = t_stop - t_start
t_final

CommClosedError: in <TCP (closed) ConnectionPool.broadcast local=tcp://10.42.0.164:39222 remote=tcp://10.42.0.165:8786>: Stream is closed

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [ ]:
print("Labels: " + str(label))

### Plotting dataimport matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)

#filter rows of original data
#label = label.to_array()
print(kmeans.labels_)
filtered_label0 = et_eller_andet[label == 0]
print(len(filtered_label0))

#Getting unique labels
u_labels = np.unique(label)